In [117]:
import importlib
import sys
from urllib.request import urlretrieve

# Install drake (and underactuated).
if 'google.colab' in sys.modules and importlib.util.find_spec('underactuated') is None:
    urlretrieve(f"http://underactuated.csail.mit.edu/scripts/setup/setup_underactuated_colab.py",
                "setup_underactuated_colab.py")
    from setup_underactuated_colab import setup_underactuated
    setup_underactuated(underactuated_sha='15cfd96b0bdfd1b0c67597c24f91907776c02a6d', drake_version='0.27.0', drake_build='release')

server_args = []
if 'google.colab' in sys.modules:
  server_args = ['--ngrok_http_tunnel']
# Start a single meshcat server instance to use for the remainder of this notebook.
from meshcat.servers.zmqserver import start_zmq_server_as_subprocess
proc_planar, zmq_url_planar, web_url_planar = start_zmq_server_as_subprocess(server_args=server_args)
proc, zmq_url, web_url = start_zmq_server_as_subprocess(server_args=server_args)

import numpy as np
from ipywidgets import FloatSlider, ToggleButton
from IPython.display import display, SVG, HTML
import pydot

import pydrake.all
from pydrake.all import (
    MultibodyPlant, DiagramBuilder, LinearQuadraticRegulator, Saturation, SceneGraph, Simulator, 
    WrapToSystem, AddMultibodyPlantSceneGraph, Parser, MathematicalProgram, eq, PiecewisePolynomial,
    SnoptSolver, TrajectorySource, MultibodyPositionToGeometryPose, PlanarSceneGraphVisualizer
)
from pydrake.examples.acrobot import AcrobotPlant, AcrobotGeometry
from pydrake.systems.jupyter_widgets import WidgetSystem
from pydrake.common.containers import namedview
from underactuated import FindResource, ManipulatorDynamics
from underactuated.jupyter import running_as_notebook


## Problem Description

We want to write a nonlinear optimization for the ball balancing bot. In order to complete our trajectory optimization, we will use `MathematicalProgram`.

**maybe write more later if we submit the notebook

## Parse the `urdf` and Get the `MultibodyPlant`

We start by defining a couple of physical parameters that we will need below.

In [118]:
ballbot_urdf = """
<?xml version="1.0"?>

<robot xmlns="http://drake.mit.edu"
 xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"
 name="BallBot">

  <link name="ground">
    <visual>
      <origin xyz="0 0 -5" rpy="0 0 0" />
      <geometry>
        <box size="1000 1000 10" />
      </geometry>
      <material>
        <color rgba="0.93 .74 .4 1" />
      </material>
    </visual>
  </link>

  <joint name="ground_weld" type="fixed">
    <parent link="world" />
    <child link="ground" />
  </joint>

  <link name="ball">
    <inertial>
      <origin xyz="0 0 0" rpy="0 0 0" />
      <mass value="5" />
      <inertia ixx=".2" ixy="0" ixz="0" iyy="0.2" iyz="0" izz="0.2" />
    </inertial>
    <visual>
      <origin xyz="0 0 0" rpy="0 0 0" />
      <geometry>
        <sphere radius=".1" />
      </geometry>
      <material>
        <color rgba="0.25 0.52 0.96 1" />
      </material>
    </visual>
  </link>

  <link name="bot">
    <inertial>
      <origin xyz="0 0 .05" rpy="0 0 0" />
      <mass value="4" />
      <inertia ixx="0.10" ixy="0" ixz="0" iyy="0.18" iyz="0" izz="0.1" />
    </inertial>
    <visual>
      <origin xyz="0 0 .05" rpy="0 0 0" />
      <geometry>
         <cylinder length=".1" radius=".12" />
      </geometry>
      <material>
        <color rgba=".61 .63 .67 1" />
      </material>
    </visual>
  </link>
  
  <joint name="x" type="prismatic">
    <parent link="world" />
    <child link="ball" />
    <origin xyz="0 0 .1" />
    <axis xyz="1 0 0" />
    <dynamics damping="0.1" />
  </joint>
   

  <joint name="theta1" type="continuous">
    <parent link="ball" />
    <child link="bot" />
    <origin xyz="0 0 0" />
    <axis xyz="0 1 0" />
    <dynamics damping="0.1" />
  </joint>


  <transmission type="SimpleTransmission" name="ball_torque">
    <actuator name="torque" />
    <joint name="theta1" />
    <mechanicalReduction>1</mechanicalReduction>
  </transmission>

  <transmission type="SimpleTransmission" name="ball_force">
    <actuator name="force" />
    <joint name="x" />
    <mechanicalReduction>.1</mechanicalReduction>
  </transmission>
</robot>
"""

## The Trajectory Optimization Problem

We start by setting some parameters of our optimization problem.

Here are part of the constraints of the optimization problem:


1.   Time steps need to be bounded between h_min and h_max
2.   Map configurations, velocities, accelerations to each other using the Euler method
3.   Something about dynamics
4.   Want the thing on top to be horizonal at the end (theta1 = 0, theta2 = 0)
5.   Can add an angle bound (like it can't be more than 15 deg away from horizontal)



Here we set the initial guess for our optimization problem.

For the time steps `h` we just initialize them to their maximal value `h_max` (somewhat an arbitrary decision, but it works).

For the robot configuration `q`, we interpolate between the initial value `q0_guess` and the final value `- q0_guess`.
In our implementation, the value given below for `q0_guess` made the optimization converge.
But, if you find the need, feel free to tweak this parameter.
The initial guess for the velocity and the acceleration is obtained by differentiating the one for the position.

The normal force `f` at the stance foot is equal to the total `weight` of the robot.

All the other optimization variables are initialized at zero.
(Note that, if the initial guess for a variable is not specified, the default value is zero.)

We can finally solve the problem! Be sure that the solver actually converged: you can check this by looking at the variable result.is_success() (printed below).

In the following cell we retrieve the optimal value of the decision variables.

## Animate the Result

Here we quickly build a Drake diagram to animate the result we got from trajectory optimization: useful for debugging your code and to be sure that everything looks good.

In [215]:
class optimization_step:
  def __init__(self, start, goal, T, actuation_lim, h_min = 0.005, 
               h_max = 0.05):
    self.start = start
    self.goal = goal
    self.T = T
    self.actuation_lim = actuation_lim
    self.h_min = h_min
    self.h_max = h_max
  #  self.max_control = max_control
  #  self.min_control = min_control

  def create_plant(self, ballbot_urdf):
    self.ballbot_2D = MultibodyPlant(time_step=0)
    Parser(self.ballbot_2D).AddModelFromString(ballbot_urdf, "urdf")
    self.ballbot_2D.Finalize()

    # overwrite MultibodyPlant with its autodiff copy
    self.ballbot_2D = self.ballbot_2D.ToAutoDiffXd() 

    # number of configuration variables
    self.nq = self.ballbot_2D.num_positions()  # Returns the size of the multibody system state vector x = [q v]. This will be num_positions() plus num_velocities()

  def ballbot2D_dynamics(self, vars):
    split_at = [self.nq, 2 * self.nq, 3 * self.nq]
    q, qd, qdd, u = np.split(vars, split_at)
    
    # set compass gait state
    context = self.ballbot_2D.CreateDefaultContext()
    self.ballbot_2D.SetPositions(context, q)
    self.ballbot_2D.SetVelocities(context, qd)
    self.ballbot_2D.get_actuation_input_port().FixValue(context, u)
    
    # matrices for the manipulator equations
    M, Cv, tauG, B, tauExt = ManipulatorDynamics(self.ballbot_2D,q,qd)
    theta = q[1]

    # return violation of the manipulator equations
    return M.dot(qdd) + Cv - tauG -B@u - tauExt

  def generate_cont_vars(self, prog):

    """ Returns h, q, qd, qdd, u """
    # vector of the time intervals
    # (distances between the T + 1 break points)
    T = self.T
    nq = self.nq
    h = prog.NewContinuousVariables(T, name='h')

    # system configuration, generalized velocities, and accelerations
    q = prog.NewContinuousVariables(rows=T+1, cols=nq, name='q')
    qd = prog.NewContinuousVariables(rows=T+1, cols=nq, name='qd')
    qdd = prog.NewContinuousVariables(rows=T, cols=nq, name='qdd')

    # actuation
    u = prog.NewContinuousVariables(rows=T, cols=nq, name='u')

    return h, q, qd, qdd, u

  def set_initial_guess(self, prog, h_max, T, h, q, qd, qdd):
    # vector of the initial guess
    initial_guess = np.empty(prog.num_vars())

    # initial guess for the time step
    h_guess = h_max
    prog.SetDecisionVariableValueInVector(h, [h_guess] * T, initial_guess)

    # linear interpolation of the configuration
    q0_guess = np.array([0, .15])
    q_guess_poly = PiecewisePolynomial.FirstOrderHold(
        [0, T * h_guess],
        np.column_stack((q0_guess, - q0_guess))
    )
    qd_guess_poly = q_guess_poly.derivative()
    qdd_guess_poly = q_guess_poly.derivative()

    # set initial guess for configuration, velocity, and acceleration
    q_guess = np.hstack([q_guess_poly.value(t * h_guess) for t in range(T + 1)]).T
    qd_guess = np.hstack([qd_guess_poly.value(t * h_guess) for t in range(T + 1)]).T
    qdd_guess = np.hstack([qdd_guess_poly.value(t * h_guess) for t in range(T)]).T
    prog.SetDecisionVariableValueInVector(q, q_guess, initial_guess)
    prog.SetDecisionVariableValueInVector(qd, qd_guess, initial_guess)
    prog.SetDecisionVariableValueInVector(qdd, qdd_guess, initial_guess)

    return initial_guess

  def get_states(self, solution):
    T = self.T
    result, h, q, qd, qdd, u = solution
    h_opt = result.GetSolution(h)
    q_opt = result.GetSolution(q)
    qd_opt = result.GetSolution(qd)
    qdd_opt = result.GetSolution(qdd)
    u_opt = result.GetSolution(u)

    # stack states
    x_opt = np.hstack((q_opt, qd_opt))

    # interpolate u values for animation
    time_breaks_opt = np.array([sum(h_opt[:t]) for t in range(T+1)])
    x_opt_poly = PiecewisePolynomial.FirstOrderHold(time_breaks_opt, x_opt.T)
    return x_opt_poly


  def simulate(self, result, h, q, qd, qdd, u):
    T = self.T
    # get optimal solution
    h_opt = result.GetSolution(h)
    q_opt = result.GetSolution(q)
    qd_opt = result.GetSolution(qd)
    qdd_opt = result.GetSolution(qdd)

    # stack states
    x_opt = np.hstack((q_opt, qd_opt))

    # interpolate state values for animation
    time_breaks_opt = np.array([sum(h_opt[:t]) for t in range(T+1)])
    x_opt_poly = PiecewisePolynomial.FirstOrderHold(time_breaks_opt, x_opt.T)

    print(f'shape of time_breaks_opt: {time_breaks_opt.shape}')
    print(time_breaks_opt)
    print(f'shape of x_opt: {x_opt.shape}')
    print(x_opt)

    # parse urdf with scene graph
    ballbot = MultibodyPlant(time_step=0)
    scene_graph = SceneGraph()
    ballbot.RegisterAsSourceForSceneGraph(scene_graph)
    Parser(ballbot).AddModelFromString(ballbot_urdf, "urdf")
    ballbot.Finalize()

    # build block diagram and drive system state with
    # the trajectory from the optimization problem
    builder = DiagramBuilder()
    source = builder.AddSystem(TrajectorySource(x_opt_poly))
    builder.AddSystem(scene_graph)
    pos_to_pose = builder.AddSystem(MultibodyPositionToGeometryPose(ballbot, input_multibody_state=True))
    builder.Connect(source.get_output_port(0), pos_to_pose.get_input_port())
    builder.Connect(pos_to_pose.get_output_port(), scene_graph.get_source_pose_port(ballbot.get_source_id()))

    # add visualizer
    xlim = [-.75, .75]
    ylim = [-.2, 1.5]
    visualizer = builder.AddSystem(PlanarSceneGraphVisualizer(scene_graph, xlim=xlim, ylim=ylim, show=False))
    builder.Connect(scene_graph.get_pose_bundle_output_port(), visualizer.get_input_port(0))
    simulator = Simulator(builder.Build())

    # generate and display animation
    visualizer.start_recording()
    simulator.AdvanceTo(x_opt_poly.end_time())
    ani = visualizer.get_recording_as_animation()
    return ani
  

    # --------------- Various Optimization Setups -----------

  def no_cost(self):
    T = self.T
    h_min = self.h_min
    h_max = self.h_max
    goal = self.goal
    start = self.start
    actuation_lim = self.actuation_lim
    nq = self.nq
    # initialize program
    prog = MathematicalProgram()
    h, q, qd, qdd, u = self.generate_cont_vars(prog)

    # 1. lower and upper bound on the time steps for all t
    prog.AddBoundingBoxConstraint([h_min] * T, [h_max] * T, h)

    # 2. link the configurations, velocities, and accelerations
    # uses implicit Euler method, https://en.wikipedia.org/wiki/Backward_Euler_method
    for t in range(T):
        prog.AddConstraint(eq(q[t+1], q[t] + h[t] * qd[t+1]))
        prog.AddConstraint(eq(qd[t+1], qd[t] + h[t] * qdd[t]))

    # 3. drake dynamics
    for t in range(T):
      vars = np.concatenate((q[t], qd[t], qdd[t], u[t]))
      prog.AddConstraint(self.ballbot2D_dynamics, lb=[0]*nq, ub=[0]*nq, vars=vars)
      prog.AddLinearConstraint(u[t][0] == u[t][1])  

    # 4. horizontal at end
    # q = [x, theta1]
    prog.AddLinearConstraint(q[-1][0] == goal[0])
    prog.AddLinearConstraint(q[-1][1] == goal[1])
    prog.AddLinearConstraint(qd[-1][0] == goal[2])
    prog.AddLinearConstraint(qd[-1][1] == goal[3])

    # give start position
    prog.AddLinearConstraint(q[0][0] == start[0])
    prog.AddLinearConstraint(q[0][1] == start[1])
    #prog.AddLinearConstraint(qd[0][0] == start[2]) #suzanna didn't have this, adding it here to include it later
    #prog.AddLinearConstraint(qd[0][1] == start[3])

    initial_guess = self.set_initial_guess(prog, h_max, T, h, q, qd, qdd)

    # solve mathematical program with initial guess
    solver = SnoptSolver()
    result = solver.Solve(prog, initial_guess)
    print(f'Solution found? {result.is_success()}.')

    return result, h, q, qd, qdd, u

  
  def distance_cost(self):
    T = self.T
    h_min = self.h_min
    h_max = self.h_max
    goal = self.goal
    start = self.start
    actuation_lim = self.actuation_lim
    nq = self.nq
    # initialize program
    prog = MathematicalProgram()
    h, q, qd, qdd, u = self.generate_cont_vars(prog)

    # 1. lower and upper bound on the time steps for all t
    prog.AddBoundingBoxConstraint([h_min] * T, [h_max] * T, h)

    # 2. link the configurations, velocities, and accelerations
    # uses implicit Euler method, https://en.wikipedia.org/wiki/Backward_Euler_method
    for t in range(T):
        prog.AddConstraint(eq(q[t+1], q[t] + h[t] * qd[t+1]))
        prog.AddConstraint(eq(qd[t+1], qd[t] + h[t] * qdd[t]))

    # 3. drake dynamics
    for t in range(T):
      vars = np.concatenate((q[t], qd[t], qdd[t], u[t]))
      prog.AddConstraint(self.ballbot2D_dynamics, lb=[0]*nq, ub=[0]*nq, vars=vars)
      prog.AddLinearConstraint(u[t][0] == u[t][1])

    # 4. Penalize time away from goal
    total = 0
    for t in range(T):
      total += (q[t][1]**2)**0.5
      total += (q[t][0]**2)**0.5
    prog.AddCost(total)  

    # 5. goal at end
    prog.AddLinearConstraint(q[-1][0] == goal[0])
    prog.AddLinearConstraint(q[-1][1] == goal[1])
    prog.AddLinearConstraint(qd[-1][0] == goal[2])
    prog.AddLinearConstraint(qd[-1][1] == goal[3])


    # give start position
    prog.AddLinearConstraint(q[0][0] == start[0])
    prog.AddLinearConstraint(q[0][1] == start[1])
    prog.AddLinearConstraint(qd[0][0] == start[2])
    prog.AddLinearConstraint(qd[0][1] == start[3])

    initial_guess = self.set_initial_guess(prog, h_max, T, h, q, qd, qdd)

    # solve mathematical program with initial guess
    solver = SnoptSolver()
    result = solver.Solve(prog, initial_guess)
    print(f'Solution found? {result.is_success()}.')

    return result, h, q, qd, qdd, u

  def quadratic_distance_cost(self):
    T = self.T
    h_min = self.h_min
    h_max = self.h_max
    goal = self.goal
    start = self.start
    actuation_lim = self.actuation_lim
    nq = self.nq
    tilt_lim = None

    epsilon = 1e-3

    # initialize program
    prog = MathematicalProgram()
    h, q, qd, qdd, u = self.generate_cont_vars(prog)

    # 1. lower and upper bound on the time steps for all t
    prog.AddBoundingBoxConstraint([h_min] * T, [h_max] * T, h)

    # 2. link the configurations, velocities, and accelerations
    # uses implicit Euler method, https://en.wikipedia.org/wiki/Backward_Euler_method
    for t in range(T):
        prog.AddConstraint(eq(q[t+1], q[t] + h[t] * qd[t+1]))
        prog.AddConstraint(eq(qd[t+1], qd[t] + h[t] * qdd[t]))

    # 3. drake dynamics
    for t in range(T):
      vars = np.concatenate((q[t], qd[t], qdd[t], u[t]))
      prog.AddConstraint(self.ballbot2D_dynamics, lb=[0]*nq, ub=[0]*nq, vars=vars)
      prog.AddLinearConstraint(u[t][0] == u[t][1])

    # 4. Penalize time away from goal
    total = 0
    for t in range(T):
      total += q[t][1]**2
      total += q[t][0]**2
    prog.AddCost(total)  

    # 5. goal at end
    #prog.AddLinearConstraint(q[-1][1] == goal[1])
    #prog.AddLinearConstraint(q[-1][0] == goal[0])
    #prog.AddLinearConstraint(qd[-1][0] == goal[2])
    #prog.AddLinearConstraint(qd[-1][1] == goal[3])
    
    prog.AddLinearConstraint((q[-1][1] - goal[1]) <= epsilon)
    prog.AddLinearConstraint((q[-1][0] - goal[0]) <= epsilon)
    prog.AddLinearConstraint((qd[-1][0] - goal[2]) <= epsilon)
    prog.AddLinearConstraint((qd[-1][1] - goal[3]) <= epsilon)

    prog.AddLinearConstraint((q[-1][1] - goal[1]) >= -epsilon)
    prog.AddLinearConstraint((q[-1][0] - goal[0]) >= -epsilon)
    prog.AddLinearConstraint((qd[-1][0] - goal[2]) >= -epsilon)
    prog.AddLinearConstraint((qd[-1][1] - goal[3]) >= -epsilon)

    # 6. give start position
    prog.AddLinearConstraint(q[0][0] == start[0])
    prog.AddLinearConstraint(q[0][1] == start[1])
    prog.AddLinearConstraint(qd[0][0] == start[2])
    prog.AddLinearConstraint(qd[0][1] == start[3])

    # 7. actuation constraint
    if actuation_lim is not None:
      for t in range(T):
        prog.AddLinearConstraint(u[t][0] <= actuation_lim)
        prog.AddLinearConstraint(u[t][0] >= -1* actuation_lim)

    # 8. Tilt Angle Constraint
    if tilt_lim is not None:
      for t in range(T):
        prog.AddLinearConstraint(q[t][1] <= tilt_lim)
        prog.AddLinearConstraint(q[t][1] >= -1* tilt_lim)

    initial_guess = self.set_initial_guess(prog, h_max, T, h, q, qd, qdd)

    # solve mathematical program with initial guess
    solver = SnoptSolver()
    result = solver.Solve(prog, initial_guess)
    print(f'Solution found? {result.is_success()}.')
    
    return result, h, q, qd, qdd, u
  
  def lqr_cost(self):
    T = self.T
    h_min = self.h_min
    h_max = self.h_max
    goal = self.goal
    start = self.start
    actuation_lim = self.actuation_lim
    nq = self.nq
    tilt_lim = None

    # initialize program
    prog = MathematicalProgram()
    h, q, qd, qdd, u = self.generate_cont_vars(prog)

    # 1. lower and upper bound on the time steps for all t
    prog.AddBoundingBoxConstraint([h_min] * T, [h_max] * T, h)

    # 2. link the configurations, velocities, and accelerations
    # uses implicit Euler method, https://en.wikipedia.org/wiki/Backward_Euler_method
    for t in range(T):
        prog.AddConstraint(eq(q[t+1], q[t] + h[t] * qd[t+1]))
        prog.AddConstraint(eq(qd[t+1], qd[t] + h[t] * qdd[t]))

    # 3. drake dynamics
    for t in range(T):
      vars = np.concatenate((q[t], qd[t], qdd[t], u[t]))
      prog.AddConstraint(self.ballbot2D_dynamics, lb=[0]*nq, ub=[0]*nq, vars=vars)
      prog.AddLinearConstraint(u[t][0] == u[t][1])

    # 4. Penalize time away from goal
    total = 0
    for t in range(T):
      total += 10*(q[t][0] - goal[0])**2
      total += 10*(q[t][1] - goal[1])**2
    prog.AddCost(total)  

    # 5. Penalize velocity away from goal
    if len(goal) == 4:
      total = 0
      for t in range(T):
        total += (q[t][0] - goal[2])**2
        total += (q[t][1] - goal[3])**2
      prog.AddCost(total)  

    # 6. Penalize actuation squared
    total = 0
    for t in range(T):
      total += u[t][0]**2
    prog.AddCost(total)  

    # 7. goal at end
    prog.AddLinearConstraint(q[-1][1] == goal[1])
    prog.AddLinearConstraint(q[-1][0] == goal[0])
    # doesn't seem to like having velocity set to zero at the end, so
    # we trust that the cost function will take care of that and don't
    # include that as a constraint here


    # 8. give start position
    prog.AddLinearConstraint(q[0][0] == start[0])
    prog.AddLinearConstraint(q[0][1] == start[1])
    prog.AddLinearConstraint(qd[0][0] == start[2])
    prog.AddLinearConstraint(qd[0][1] == start[3])

    # 9. actuation constraint
    if actuation_lim is not None:
      for t in range(T):
        vars = np.concatenate((q[t], qd[t], qdd[t], u[t]))
        prog.AddLinearConstraint(u[t][0] <= actuation_lim)
        prog.AddLinearConstraint(u[t][0] >= -1* actuation_lim)

    # 10. Tilt Angle Constraint
    if tilt_lim is not None:
      for t in range(T):
        prog.AddLinearConstraint(q[t][1] <= tilt_lim)
        prog.AddLinearConstraint(q[t][1] >= -1* tilt_lim)

    initial_guess = self.set_initial_guess(prog, h_max, T, h, q, qd, qdd)

    # solve mathematical program with initial guess
    solver = SnoptSolver()
    result = solver.Solve(prog, initial_guess)

    print(f'Solution found? {result.is_success()}.')

    return result, h, q, qd, qdd, u

In [216]:
start = [-.5, -.7, .25 , .1]
goal = [0, 0, 0, 0]
T = 50
actuation_lim = None

step = optimization_step(start, goal, T, actuation_lim)
step.create_plant(ballbot_urdf)
solution = step.lqr_cost()
states = step.get_states(solution)
#states = np.array([states.value(i) for i in range(T+1)])
#print(states)
print(states.value(0))

HTML(step.simulate(*solution).to_jshtml())

Solution found? True.
[[-0.5 ]
 [-0.7 ]
 [ 0.25]
 [ 0.1 ]]
shape of time_breaks_opt: (51,)
[0.         0.05       0.1        0.15       0.2        0.25
 0.3        0.35       0.4        0.45       0.5        0.55
 0.6        0.65       0.7        0.75       0.8        0.85
 0.9        0.95       1.         1.05       1.1        1.15
 1.2        1.25       1.3        1.35       1.4        1.45
 1.5        1.55       1.56119868 1.56619868 1.57119868 1.57619868
 1.58119868 1.58619868 1.59119868 1.59619868 1.60119868 1.60619868
 1.61119868 1.61619868 1.62119868 1.62619868 1.63119868 1.63619868
 1.64119868 1.64619868 1.65119868]
shape of x_opt: (51, 4)
[[-5.00000000e-01 -7.00000000e-01  2.50000000e-01  1.00000000e-01]
 [-4.87034307e-01 -6.67973408e-01  2.59313867e-01  6.40531842e-01]
 [-4.73664464e-01 -6.18741528e-01  2.67396848e-01  9.84637603e-01]
 [-4.59954472e-01 -5.59803247e-01  2.74199843e-01  1.17876561e+00]
 [-4.45956062e-01 -4.96723645e-01  2.79968202e-01  1.26159205e+00]
 [-4.3170

In [227]:
def model_predictive_control(start, goal, optimizer, time_steps, horizon, state_noise, actuator_noise, actuation_lim = None):
  # 1. take current state (or initial condition)
  # 2. solve optimization problem
  # 3. take first commanded u from problem
  # 3. add actuator noise
  # 4. execute for a step
  # 5. add state noise
  
  nq = 4

  #result = np.zeros((nq, time_steps  + 1))
  #result[:,0] = start

  #states = start
  states = [start]
  time_intervals = []
  for i in range(time_steps):
    print(f'Beginning step {i} of {time_steps}...')

    # run trajectory optimizer with given initial conditions
    step = optimization_step(states[-1], goal, horizon, actuation_lim)
    step.create_plant(ballbot_urdf)
    solution = optimizer(step)
    result, h, q, qd, qdd, u = solution

    # since the trajectory optimizer only works with discrete states in time,
    # it has no concept of time, and so uses the h[] decision variables to figure
    # how much physical time each timestep corresponds to. this is for numerical
    # convergence (i think) but it means we have the option of running the controller
    # at uniform points with variable spacing, or at uniform times. here, we choose states.
    #x_opt_poly = step.get_states(solution)
    #next_state = x_opt_poly.value(1).squeeze()
    q_opt = result.GetSolution(q)
    qd_opt = result.GetSolution(qd)
    h_opt = result.GetSolution(h)
    next_state = [q_opt[1][0], q_opt[1][1], qd_opt[1][0], qd_opt[1][1]]

    time_intervals.append(h_opt[0]) #don't know if this should be 0 or 1 tbh
    
  
    # print what's happenin so fischer can know where to direct his range
    print(f'step size: {time_intervals[-1]}')
    print(f'current state is: {states[-1]}')
    print(f'next state is: {next_state}\n')

    states.append(next_state)

    if i == 40:
      return np.vstack(states), time_intervals


In [228]:
start = [-0.5, -0.7, 0.25, 0.1]
goal = [0, 0, 0, 0]
optimizer = optimization_step.lqr_cost
time_steps = 50
horizon = 40
state_noise = np.random.rand
actuator_noise = np.random.rand

mpc_result = model_predictive_control(start, goal, optimizer, time_steps, horizon, state_noise, actuator_noise)

Beginning step 0 of 50...
Solution found? True.
step size: 0.05
current state is: [-0.5, -0.7, 0.25, 0.1]
next state is: [-0.48703434991217326, -0.6679837390611066, 0.2593130017565347, 0.6403252187778663]

Beginning step 1 of 50...
Solution found? True.
step size: 0.05
current state is: [-0.48703434991217326, -0.6679837390611066, 0.2593130017565347, 0.6403252187778663]
next state is: [-0.47366456738992624, -0.6187693852773206, 0.26739565044596697, 0.9842870756757195]

Beginning step 2 of 50...
Solution found? True.
step size: 0.05
current state is: [-0.47366456738992624, -0.6187693852773206, 0.26739565044596697, 0.9842870756757195]
next state is: [-0.4599546170661934, -0.5598517290926494, 0.274199006474998, 1.178353123693425]

Beginning step 3 of 50...
Solution found? True.
step size: 0.05
current state is: [-0.4599546170661934, -0.5598517290926494, 0.274199006474998, 1.178353123693425]
next state is: [-0.44595620677654535, -0.49679162798150384, 0.2799682057929603, 1.2612020222229108]


In [229]:
def mpc_simulator():
  states, time_intervals = mpc_result
  time_intervals = np.array(time_intervals)
  time_breaks_opt = np.array([sum(time_intervals[:t]) for t in range(42)])
  print(f'shape of time_breaks_opt: {time_breaks_opt.shape}')
  print(time_breaks_opt)
  print(f'shape of states: {states.shape}')
  print(states)
  x_opt_poly = PiecewisePolynomial.FirstOrderHold(time_breaks_opt, states.T)

  # parse urdf with scene graph
  ballbot = MultibodyPlant(time_step=0)
  scene_graph = SceneGraph()
  ballbot.RegisterAsSourceForSceneGraph(scene_graph)
  Parser(ballbot).AddModelFromString(ballbot_urdf, "urdf")
  ballbot.Finalize()

  # build block diagram and drive system state with
  # the trajectory from the optimization problem
  builder = DiagramBuilder()
  source = builder.AddSystem(TrajectorySource(x_opt_poly))
  builder.AddSystem(scene_graph)
  pos_to_pose = builder.AddSystem(MultibodyPositionToGeometryPose(ballbot, input_multibody_state=True))
  builder.Connect(source.get_output_port(0), pos_to_pose.get_input_port())
  builder.Connect(pos_to_pose.get_output_port(), scene_graph.get_source_pose_port(ballbot.get_source_id()))

  # add visualizer
  xlim = [-.75, .75]
  ylim = [-.2, 1.5]
  visualizer = builder.AddSystem(PlanarSceneGraphVisualizer(scene_graph, xlim=xlim, ylim=ylim, show=False))
  builder.Connect(scene_graph.get_pose_bundle_output_port(), visualizer.get_input_port(0))
  simulator = Simulator(builder.Build())

  # generate and display animation
  visualizer.start_recording()
  simulator.AdvanceTo(x_opt_poly.end_time())
  ani = visualizer.get_recording_as_animation()
  return ani

In [230]:
HTML(mpc_simulator().to_jshtml())

shape of time_breaks_opt: (42,)
[0.         0.05       0.1        0.15       0.2        0.25
 0.3        0.35       0.4        0.45       0.5        0.55
 0.6        0.65       0.7        0.75       0.8        0.85
 0.9        0.92714198 0.9515932  0.9801087  1.0301087  1.05556536
 1.08412224 1.13412224 1.16930642 1.21930642 1.26930642 1.31930642
 1.36930642 1.41930642 1.46930642 1.51930642 1.56930642 1.61930642
 1.66930642 1.71930642 1.76930642 1.81930642 1.86930642 1.91930642]
shape of states: (42, 4)
[[-5.00000000e-01 -7.00000000e-01  2.50000000e-01  1.00000000e-01]
 [-4.87034350e-01 -6.67983739e-01  2.59313002e-01  6.40325219e-01]
 [-4.73664567e-01 -6.18769385e-01  2.67395650e-01  9.84287076e-01]
 [-4.59954617e-01 -5.59851729e-01  2.74199006e-01  1.17835312e+00]
 [-4.45956207e-01 -4.96791628e-01  2.79968206e-01  1.26120202e+00]
 [-4.31707759e-01 -4.33564881e-01  2.84968961e-01  1.26453494e+00]
 [-4.17238416e-01 -3.72886996e-01  2.89386848e-01  1.21355770e+00]
 [-4.02572371e-01 -3.1

In [231]:
print(mpc_result[0])
print(mpc_result[1])

[[-5.00000000e-01 -7.00000000e-01  2.50000000e-01  1.00000000e-01]
 [-4.87034350e-01 -6.67983739e-01  2.59313002e-01  6.40325219e-01]
 [-4.73664567e-01 -6.18769385e-01  2.67395650e-01  9.84287076e-01]
 [-4.59954617e-01 -5.59851729e-01  2.74199006e-01  1.17835312e+00]
 [-4.45956207e-01 -4.96791628e-01  2.79968206e-01  1.26120202e+00]
 [-4.31707759e-01 -4.33564881e-01  2.84968961e-01  1.26453494e+00]
 [-4.17238416e-01 -3.72886996e-01  2.89386848e-01  1.21355770e+00]
 [-4.02572371e-01 -3.16486775e-01  2.93320897e-01  1.12800441e+00]
 [-3.87731796e-01 -2.65345246e-01  2.96811506e-01  1.02283058e+00]
 [-3.72738194e-01 -2.19895825e-01  2.99872039e-01  9.08988420e-01]
 [-3.57612639e-01 -1.80184077e-01  3.02511108e-01  7.94234972e-01]
 [-3.42375445e-01 -1.45995179e-01  3.04743881e-01  6.83777947e-01]
 [-3.27045682e-01 -1.16949719e-01  3.06595264e-01  5.80909199e-01]
 [-3.11640761e-01 -9.25674887e-02  3.08098404e-01  4.87644610e-01]
 [-2.96176158e-01 -7.23380819e-02  3.09292069e-01  4.04588136e